I finished in the  top 8% and earned the bronze medal by applying some post-processing. Though I would have preferred to train my own model, I did not--whenever I tried any that would have had a chance, I received out of memory messages, plus I am a beginner and it's unlikely I would have exceeded the performance of the ones made public. So I started with <a href='https://www.kaggle.com/code/abhishek/two-longformers-are-better-than-1'>Abhishek Thakur's notebook</a>, and other than changing a few hyperparameters (which boosted the public score by .01), I made no changes.  

The post-processing used below increased the public score another .03, which I believe lifted the entry to the bronze medal.  

Below, I don't include any of the model or model application code; it is available in the linked notebook and on Abhishek's GitHub repository which is linked to from his Kaggle notebook. Instead just to illustrate I use a sample that was generated in the course of testing the post-procecessing.

In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

INPUT_PATH="../input/feedback-prize-2021/"
SUBMISSION_SAMPLE="../input/predictionsfromtez/fold0_preds"

In [ ]:
def link_consecutive(submission, class_name):
    #limit to class with conesecutive entries
    df=submission.copy()
    preds_class=df[df["class"]==class_name]
    consec_class = preds_class.query(f"nextclass=='{class_name}' and lastword_index +1 ==nextfirstwordindex")
    ids=consec_class.id.unique()
    preds_class_to_review=preds_class[preds_class.id.isin(ids)]
    retval=[]
    for i,r in preds_class_to_review.iterrows():
        if r["nextclass"]==class_name and r["lastword_index"] +1 ==r["nextfirstwordindex"]:
            newpredstring=r["predictionstring"] + " " + r["nextpredictionstring"]
            retval.append((r["id"], r["class"], newpredstring,r['firstword_index'],r['nextlastwordindex']))
            df=df.drop(df[(df["id"]==r.id) & (df["class"]==class_name) & (df["firstword_index"].astype(int)==int(newpredstring.split()[0]))].index)
            df=df.drop(df[(df["id"]==r.id) & (df["class"]==class_name) & (df["lastword_index"].astype(int)==int(newpredstring.split()[len(newpredstring.split())-1]))].index)        

    retval_df=pd.DataFrame(retval)
    
    if(retval_df.empty==False):
        retval_df.columns=['id', 'class', 'predictionstring', 'firstword_index', 'lastword_index']
        df=df.merge(retval_df, how='outer')
        #df.head()
        df.sort_values(["id", "firstword_index"],inplace=True)
        df["predstringsplit"]=df["predictionstring"].apply(lambda x:x.split())
        df['nextclass'] = df['class'].shift(-1)
        df['nextfirstwordindex'] = df['firstword_index'].shift(-1)
        df['nextlastwordindex'] = df['lastword_index'].shift(-1)
        df['nextpredictionstring']= df['predictionstring'].shift(-1)
    
    return(df)


In [ ]:
def link_near_consecutive(df, class_name, distance=8):

    preds_class=df.query(f"`class`=='{class_name}'")
    consec_class = preds_class.query(f"nextclass=='{class_name}'")
    consec_class=consec_class[consec_class["nextfirstwordindex"]-distance<=consec_class["lastword_index"]]                            
    ids=consec_class.id.unique()
    preds_class_to_review=preds_class[preds_class.id.isin(ids)]
    retval=[]
    for i,r in preds_class_to_review.iterrows():
        if r["nextclass"]==class_name and r["nextfirstwordindex"] - distance <=r["lastword_index"]:
            intermediate = range(int(r["lastword_index"]+1), int(r["nextfirstwordindex"]))
            intermediate=" ".join([str(item) for item in intermediate])
            newpredstring=r["predictionstring"] + " " + intermediate +  " " + r["nextpredictionstring"]
            retval.append((r["id"], r["class"], newpredstring,r['firstword_index'],r['nextlastwordindex']))
            df=df.drop(df[(df["id"]==r.id) & (df["class"]==class_name) & (df["firstword_index"].astype(int)==int(newpredstring.split()[0]))].index)
            df=df.drop(df[(df["id"]==r.id) & (df["class"]==class_name) & (df["lastword_index"].astype(int)==int(newpredstring.split()[len(newpredstring.split())-1]))].index)        

    retval_df=pd.DataFrame(retval)
    if(retval_df.empty==False):
        retval_df.columns=['id', 'class', 'predictionstring', 'firstword_index', 'lastword_index']
        df=df.merge(retval_df, how='outer')
        df.sort_values(["id", "firstword_index"],inplace=True)
        df['nextclass'] = df['class'].shift(-1)
        df['nextfirstwordindex'] = df['firstword_index'].shift(-1)
        df['nextlastwordindex'] = df['lastword_index'].shift(-1)
        df['nextpredictionstring']= df['predictionstring'].shift(-1)
    return df

In [ ]:
#since this is just an illustration, choose a few records to speed things up
submission = pd.read_csv(SUBMISSION_SAMPLE).head(500)


Temporariliy add additional features to enable the post-processing:

In [ ]:
submission["predstringsplit"]=submission["predictionstring"].apply(lambda x:x.split())
submission["firstword_index"]=submission["predstringsplit"].apply(lambda x:int(x[0]))
submission["lastword_index"]=submission["predstringsplit"].apply(lambda x:int(x[len(x)-1]))
submission.sort_values(["id", "firstword_index"],inplace=True)
submission['nextclass'] = submission['class'].shift(-1)
submission['nextfirstwordindex'] = submission['firstword_index'].shift(-1)
submission['nextlastwordindex'] = submission['lastword_index'].shift(-1)
submission['nextpredictionstring']= submission['predictionstring'].shift(-1)

Combine consecutive passage with the same label, e.g., 

Lead 1,2,3,4
Lead 5,6,7,8

Becomes 

Lead 1,2,3,4,5,6,7,8

In [ ]:
submission=link_consecutive(submission, 'Rebuttal')
submission=link_consecutive(submission, 'Counterclaim')
submission=link_consecutive(submission, 'Lead')
submission=link_consecutive(submission, 'Concluding Statement')
submission=link_consecutive(submission, 'Position')

Below combines almost consecutive passages if they are separated by 4 words or less by another passage with no label, e.g,, 

Rebuttal 20,21,22,23
Rebuttal 26,27,28,29,30

Becomes

Rebuttal 20,21,22,23,24,25,26,27,28,29,30

As long as there is no label for 24,25

In [ ]:
submission=link_near_consecutive(submission, 'Rebuttal',4)
submission=link_near_consecutive(submission, 'Counterclaim',4)
submission=link_near_consecutive(submission, 'Lead',4)
submission=link_near_consecutive(submission, 'Concluding Statement')
#submission=link_near_consecutive(submission, 'Position')

Add more features to enable the next heurestic rule:

In [ ]:
test_dict={}
ids = submission.id.unique()

for id in ids:
        filename = f"{INPUT_PATH}train/{id}.txt"
        with open(filename, "r") as f:
            text = f.read()
            test_dict[id]=text


In [ ]:
submission["predstringsplit"]=submission["predictionstring"].apply(lambda x:x.split())
submission["firstword_index"]=submission["predstringsplit"].apply(lambda x:int(x[0]))
submission["lastword_index"]=submission["predstringsplit"].apply(lambda x:int(x[len(x)-1]))
submission.sort_values(["id", "firstword_index"],inplace=True)
submission['nextclass'] = submission['class'].shift(-1)
submission['nextfirstwordindex'] = submission['firstword_index'].shift(-1)
submission['nextlastwordindex'] = submission['lastword_index'].shift(-1)
submission['nextpredictionstring']= submission['predictionstring'].shift(-1)

In [ ]:

submission["prediction_text"]=submission.apply(lambda x:" ".join(test_dict.get(x.id).split()[int(x.firstword_index):int(x.lastword_index)+1]), axis=1)
submission=submission[["id", "class", "predictionstring","firstword_index", "lastword_index", "nextfirstwordindex"]]

Adds the label "Nothing" for passages with no label:

In [ ]:
nothing =submission.query("nextfirstwordindex !=lastword_index+1 and nextfirstwordindex>lastword_index") 
nothing.sort_values(["id", "firstword_index"],inplace=True)
insert_row=len(nothing)
for i,r in nothing.iterrows():
     id = r["id"]
     classname="Nothing"
     intermediate = range(int(r["lastword_index"]+1), int(r["nextfirstwordindex"]))
     predstring=" ".join([str(item) for item in intermediate])
     predstringsplit=predstring.split()
     firstwordindex=int(r["lastword_index"]+1)
     lastwordindex=firstwordindex + len(predstringsplit)-1 #check
     nextfirstwordindex=lastwordindex +1
     nothing.loc[insert_row] = [id, classname,predstring, firstwordindex, lastwordindex, nextfirstwordindex]
     insert_row += 1
submission=submission.merge(nothing, how='outer')
submission.sort_values(["id", "firstword_index"],inplace=True)

In [ ]:
def getWordsFromPhrase(Phrase, startWord, EndWord):
    #print(Phrase)
    return " ".join(Phrase.split()[startWord:EndWord])


In [ ]:
submission["prediction_text"]=submission.apply(lambda x:" ".join(test_dict.get(x.id).split()[int(x.firstword_index):int(x.lastword_index)+1]), axis=1)
submission["prediction_text"]=submission["prediction_text"].str.upper()
submission["first_word"]=submission.apply(lambda x: getWordsFromPhrase(x['prediction_text'],0,1).upper(), axis=1)
submission["first_two_words"]=submission.apply(lambda x: getWordsFromPhrase(x['prediction_text'],0,2).upper(), axis=1)

I tried a number of rules like the below, which change a Nothing value to 'Claim' based on the first two words of the passage, but of the ones I tried only these were found to help.

In [ ]:
submission["class"] = submission.apply(lambda x: 'Claim' if x['first_two_words']=="YOU WILL" and x['class']=='Nothing'   else x['class'], axis=1)
submission["class"] = submission.apply(lambda x: 'Claim' if x['first_word']=="YOU'LL" and x['class']=='Nothing'   else x['class'], axis=1)

In [ ]:
#drop added features and submit
submission=submission.query("`class` !='Nothing'")
submission=submission[["id", "class", "predictionstring"]]
#submission.to_csv("submission.csv", index=False)

In [ ]:
submission